In [ ]:
#hide
# default_exp core
from nbdev.showdoc import *

# kicad-helpers

> Scripts, templates, and examples for managing KiCad projects.

In [ ]:
#export
import glob
import os
import subprocess

import git
from yaml import load, dump
try:
    from yaml import CLoader as Loader, CDumper as Dumper
except ImportError:
    from yaml import Loader, Dumper

In [ ]:
#export
def get_project_root():
    # Find the current projects' root directory
    git_repo = git.Repo(".", search_parent_directories=True)
    return git_repo.git.rev_parse("--show-toplevel")

In [ ]:
#export
# Set the PROJECT ROOT environment variable
PROJECT_ROOT = get_project_root()
os.environ["KH_PROJECT_ROOT"] = PROJECT_ROOT
assert(os.getenv("KH_PROJECT_ROOT") == PROJECT_ROOT)

In [ ]:
%%sh
echo $KH_PROJECT_ROOT

/mnt/c/Users/ryan/OneDrive/dev/kicad/kicad-helpers


The `$KH_PROJECT_ROOT` environment variable gets automatically set when the kicad_helpers package is imported making it accessible to shell scripts. `PROJECT_ROOT` is also available as a python variable in the `kicad_helpers` module.

# Project metadata/configuration

Metadata and configuration for the project are stored in the `.kicad_helpers_config` directory (including [kibot](https://github.com/INTI-CMNB/KiBot) config files for producting manufacturing outputs via continuous integration). Additional metadata (e.g., project summary, website, and manufacturing details) is stored in the project's `kitspace.yaml` file which makes it easy for other people to manufacture boards using https://kitspace.org/.

In [ ]:
#export
def get_project_name():
    # This is based on the name of the KiCad `*.pro` file.
    root = get_project_root()
    return os.path.splitext(os.path.split(glob.glob(os.path.join(root, "*.pro"))[0])[1])[0]

def get_project_metadata():
    PROJECT_ROOT = get_project_root()
    
    # Default metadata if there's no existing `kicad.yaml` file.
    metadata = {"summary": "A description for your project",
                "site": "https://example.com # A site you would like to link to (include http:// or https://)",
                "color": "black"
    }
    
    try:
        # If there's an existing `kicad.yaml` file, those settings override the defaults.
        with open(os.path.join(PROJECT_ROOT, "kitspace.yaml")) as f:
            metadata.update({k: v for k, v in load(f, Loader=Loader).items() if k in ["summary", "site", "color"]})
    except FileNotFoundError:
        pass

    # Add the project name
    metadata["project_name"]=get_project_name()
    return metadata

In [ ]:
get_project_metadata()

{'summary': 'A description for your project',
 'site': 'https://example.com',
 'color': 'black',
 'project_name': '40-channel-hv-switching-board'}

In [ ]:
#export
def get_gitignore_list():
    PROJECT_ROOT = get_project_root()
    with open(f"{PROJECT_ROOT}/.gitignore") as f:
        gitignore = f.readlines()
    return "|".join([line.strip() for line in gitignore])

In [ ]:
#export
def in_gitignore(filename):
    try:
        if len(subprocess.check_output(f"echo '{ filename }' | git check-ignore --stdin --no-index", shell=True)):
            return True
    except subprocess.CalledProcessError:
        pass
    return False